In [1]:
import os.path as osp
import random
import time

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data

In [2]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
from utils.ssd_model import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list, od_collate_fn


rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath)

# Dataset
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123) 
input_size = 300  

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))


# DataLoader
batch_size = 16

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn)

dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


In [5]:
from utils.ssd_model import SSD

# SSD300
ssd_cfg = {
    'num_classes': 21,  
    'input_size': 300,  
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  
    'feature_maps': [38, 19, 10, 5, 3, 1],  
    'steps': [8, 16, 32, 64, 100, 300],  
    'min_sizes': [30, 60, 111, 162, 213, 264],  #
    'max_sizes': [60, 111, 162, 213, 264, 315],  
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSD
net = SSD(phase="train", cfg=ssd_cfg)

# SSD
vgg_weights = torch.load('./weights/vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights)

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:  
            nn.init.constant_(m.bias, 0.0)

net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
from utils.ssd_model import MultiBoxLoss

criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3, device=device)

optimizer = optim.SGD(net.parameters(), lr=1e-3,
                      momentum=0.9, weight_decay=5e-4)


In [7]:

def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)

    net.to(device)

    torch.backends.cudnn.benchmark = True

    iteration = 1
    epoch_train_loss = 0.0  
    epoch_val_loss = 0.0  
    logs = []

    for epoch in range(num_epochs+1):

        t_epoch_start = time.time()
        t_iter_start = time.time()

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  
                print('（train）')
            else:
                if((epoch+1) % 10 == 0):
                    net.eval()   
                    print('-------------')
                    print('（val）')
                else:
                    
                    continue

            for images, targets in dataloaders_dict[phase]:

                images = images.to(device)
                targets = [ann.to(device)
                           for ann in targets]  

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(images)

                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c

                    if phase == 'train':
                        loss.backward()  

                        nn.utils.clip_grad_value_(
                            net.parameters(), clip_value=2.0)

                        optimizer.step()  

                        if (iteration % 10 == 0):  
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('{} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item(), duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    else:
                        epoch_val_loss += loss.item()

        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

        epoch_train_loss = 0.0  
        epoch_val_loss = 0.0  

        if ((epoch+1) % 10 == 0):
            torch.save(net.state_dict(), 'weights/ssd300_' +
                       str(epoch+1) + '.pth')


In [9]:
num_epochs= 15  
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

cuda:0
-------------
Epoch 1/15
-------------
（train）
イテレーション 10 || Loss: 6.8240 || 10iter: 5.7025 sec.
イテレーション 20 || Loss: 8.9419 || 10iter: 5.7785 sec.
イテレーション 30 || Loss: 8.4657 || 10iter: 5.7064 sec.
イテレーション 40 || Loss: 6.9927 || 10iter: 5.9391 sec.
イテレーション 50 || Loss: 8.1591 || 10iter: 5.7091 sec.
イテレーション 60 || Loss: 7.0352 || 10iter: 5.7091 sec.
イテレーション 70 || Loss: 7.6408 || 10iter: 5.6092 sec.
イテレーション 80 || Loss: 7.6531 || 10iter: 5.8045 sec.
イテレーション 90 || Loss: 7.0336 || 10iter: 5.8602 sec.
イテレーション 100 || Loss: 7.8838 || 10iter: 6.1169 sec.
イテレーション 110 || Loss: 7.9652 || 10iter: 5.5233 sec.
イテレーション 120 || Loss: 7.6176 || 10iter: 6.0152 sec.
イテレーション 130 || Loss: 7.1404 || 10iter: 6.0684 sec.
イテレーション 140 || Loss: 7.2827 || 10iter: 5.7819 sec.
イテレーション 150 || Loss: 7.3701 || 10iter: 5.6986 sec.
イテレーション 160 || Loss: 6.9449 || 10iter: 5.6960 sec.
イテレーション 170 || Loss: 6.8070 || 10iter: 5.8830 sec.
イテレーション 180 || Loss: 7.1349 || 10iter: 5.7723 sec.
イテレーション 190 || Loss: 6.5871 || 10iter

イテレーション 1490 || Loss: 6.1144 || 10iter: 5.5928 sec.
イテレーション 1500 || Loss: 6.2383 || 10iter: 5.4208 sec.
イテレーション 1510 || Loss: 6.0608 || 10iter: 5.6361 sec.
イテレーション 1520 || Loss: 5.3633 || 10iter: 5.6264 sec.
イテレーション 1530 || Loss: 4.9877 || 10iter: 5.4152 sec.
イテレーション 1540 || Loss: 5.1325 || 10iter: 5.4996 sec.
イテレーション 1550 || Loss: 5.5781 || 10iter: 5.5782 sec.
イテレーション 1560 || Loss: 5.9535 || 10iter: 5.4015 sec.
イテレーション 1570 || Loss: 6.4916 || 10iter: 5.3983 sec.
イテレーション 1580 || Loss: 6.7221 || 10iter: 5.5001 sec.
イテレーション 1590 || Loss: 6.1139 || 10iter: 5.4915 sec.
イテレーション 1600 || Loss: 5.6949 || 10iter: 5.5235 sec.
イテレーション 1610 || Loss: 5.8789 || 10iter: 5.5605 sec.
イテレーション 1620 || Loss: 6.6314 || 10iter: 5.5981 sec.
イテレーション 1630 || Loss: 5.3318 || 10iter: 5.4254 sec.
イテレーション 1640 || Loss: 5.6328 || 10iter: 5.4025 sec.
イテレーション 1650 || Loss: 5.0497 || 10iter: 5.7655 sec.
イテレーション 1660 || Loss: 5.8468 || 10iter: 5.6674 sec.
イテレーション 1670 || Loss: 5.7825 || 10iter: 5.8083 sec.
イテレーション 1680

イテレーション 2960 || Loss: 5.2223 || 10iter: 5.4456 sec.
イテレーション 2970 || Loss: 4.9403 || 10iter: 5.4397 sec.
イテレーション 2980 || Loss: 5.7764 || 10iter: 5.3540 sec.
イテレーション 2990 || Loss: 5.2923 || 10iter: 5.3438 sec.
イテレーション 3000 || Loss: 5.1743 || 10iter: 5.3800 sec.
イテレーション 3010 || Loss: 5.0319 || 10iter: 5.3336 sec.
イテレーション 3020 || Loss: 5.3544 || 10iter: 5.5605 sec.
イテレーション 3030 || Loss: 5.5245 || 10iter: 5.4490 sec.
イテレーション 3040 || Loss: 5.1506 || 10iter: 5.3779 sec.
イテレーション 3050 || Loss: 4.2689 || 10iter: 5.3724 sec.
イテレーション 3060 || Loss: 5.1897 || 10iter: 5.3712 sec.
イテレーション 3070 || Loss: 5.4704 || 10iter: 5.3743 sec.
イテレーション 3080 || Loss: 5.3280 || 10iter: 5.5042 sec.
イテレーション 3090 || Loss: 4.7455 || 10iter: 5.2428 sec.
イテレーション 3100 || Loss: 4.7627 || 10iter: 5.4171 sec.
イテレーション 3110 || Loss: 4.2947 || 10iter: 5.4639 sec.
イテレーション 3120 || Loss: 5.9644 || 10iter: 5.4854 sec.
イテレーション 3130 || Loss: 5.0770 || 10iter: 5.5554 sec.
イテレーション 3140 || Loss: 5.0481 || 10iter: 5.5265 sec.
イテレーション 3150

イテレーション 4420 || Loss: 3.9511 || 10iter: 5.4563 sec.
イテレーション 4430 || Loss: 5.5623 || 10iter: 5.4862 sec.
イテレーション 4440 || Loss: 5.2141 || 10iter: 5.5780 sec.
イテレーション 4450 || Loss: 5.5026 || 10iter: 5.5961 sec.
イテレーション 4460 || Loss: 5.1174 || 10iter: 5.4921 sec.
イテレーション 4470 || Loss: 5.3002 || 10iter: 5.4803 sec.
イテレーション 4480 || Loss: 3.9576 || 10iter: 5.6106 sec.
イテレーション 4490 || Loss: 4.6991 || 10iter: 5.5198 sec.
イテレーション 4500 || Loss: 4.5371 || 10iter: 5.6288 sec.
イテレーション 4510 || Loss: 5.1872 || 10iter: 5.5479 sec.
イテレーション 4520 || Loss: 4.6645 || 10iter: 5.4564 sec.
イテレーション 4530 || Loss: 5.3153 || 10iter: 5.4631 sec.
イテレーション 4540 || Loss: 4.7308 || 10iter: 5.4457 sec.
イテレーション 4550 || Loss: 5.3028 || 10iter: 5.6165 sec.
イテレーション 4560 || Loss: 4.7774 || 10iter: 5.4632 sec.
イテレーション 4570 || Loss: 4.5458 || 10iter: 5.3466 sec.
イテレーション 4580 || Loss: 4.3161 || 10iter: 5.4435 sec.
イテレーション 4590 || Loss: 4.6702 || 10iter: 5.4138 sec.
イテレーション 4600 || Loss: 4.7969 || 10iter: 5.4487 sec.
イテレーション 4610